# Simple example: Transformations from SQL to parametrized circuit

## Parsing

We parse the queries with ANTRL framework which is an extensive general-purpose parsing tool. We selected the SQLite grammar for its simplicity in ANTRL prewritten grammars. Because the core features of SQL are the same for any relational database, this code does not depend on system's SQL dialect.

So far, the module can deal with queries that have a SELECT-FROM-WHERE structure without substatements.

In [1]:
from antlr4 import *
from SQLiteLexer import SQLiteLexer
from SQLiteParser import SQLiteParser
from SQLiteParserListener import SQLiteParserListener
import json
import os
import glob
import random
from pathlib import Path
from discopy import Ty, Box, Functor, Id, Swap, hypergraph, Cup
from functools import reduce
import numpy as np
import sympy
from discopy.quantum.pennylane import to_pennylane, PennyLaneCircuit
from sympy import default_sort_key
import pennylane as qml
import torch
#from lambeq import IQPAnsatz
from flipped_IQPansatz import IQPAnsatzFlipped
from pennylane.drawer import draw,draw_mpl,tape_mpl
from discopy.utils import dumps, loads

input_file = "simple_examples/cat.sql"

input_stream = FileStream(input_file)
lexer = SQLiteLexer(input_stream)
stream = CommonTokenStream(lexer)
parser = SQLiteParser(stream)
tree = parser.parse()

print("Whole parse tree: ")
print(tree.toStringTree(recog=parser))

Whole parse tree: 
(parse (sql_stmt_list (sql_stmt (select_stmt (select_core SELECT (result_column (expr (column_name (any_name cat_name)))) , (result_column (expr (column_name (any_name favourite_food)))) FROM (table_or_subquery (table_name (any_name cats))) WHERE (expr (expr (column_name (any_name cat_name))) = (expr (literal_value 'Whiskers')))))) ;) <EOF>)


## Diagrammatic representation of SQL language elements in context free grammar

We walk the tree and collect the elements in a monoidal category. This creates a diagrammatic representation for the parsed SQL query. `SQLiteParserListener` class implements DisCoPy construction. First we collect the abstract parse tree but we also modify it so that we collect SELECT and the columns into the same element, FROM and tables into the same element and WHERE and the filtering clauses into the same element. This interpretation follows the [railroad diagram representation](https://www.sqlite.org/syntaxdiagrams.html#select-core) more accurately than the current parsing. It also enables us to map the parse tree correctly later.

In [2]:
this_folder = os.path.abspath(os.getcwd())
total_dim = 0
walker = ParseTreeWalker()
listener = SQLiteParserListener(parser)
walker.walk(listener, tree)
diagram = listener.get_final_diagram()
width = diagram.width()
height = diagram.depth()
dim = 4*max(width, height)
total_dim += dim

In [3]:
# figsize=(30, 20), fontsize = 17, fontsize_types = 17, scale = (0.8, 1),
diagram.draw(path = this_folder + "\\figures\\paper_figures\\CFG_diagram_cat_example.tex", to_tikz = True)

## Map context free grammar representations to pregroup representations

Because the abstract syntax tree contains lots of unnecessary information for our purposes, we will functorially rewrite it. This functorial rewriting process is just something that we have developed especially for this work and it is open to discussion if there exists a more suitable rewrite mapping. Also, the parts `select_main`, `from_main` and `where_main` are especially designed for this work although they are visible in the SQLite railroad diagrams without any special name.

Functor simply describes how the boxes and types are mapped. In this rewriting process we want to simplify the abstract syntax tree.

In [4]:
#from cfg_alias_rewriting_mappings import alias_object_mapping, alias_morphism_mapping

#Rewriter = Functor(ob = lambda x: alias_object_mapping(x), ar = lambda f: alias_morphism_mapping(f))
#cfg_diagram = Rewriter(diagram)
#cfg_diagram.draw(figsize=(15, 15)) #, path = this_folder + "\\figures\\pregroup_figure.png")

#back_n_forth = lambda f: hypergraph.Diagram.upgrade(f).downgrade()
#back_n_forth(cfg_diagram).draw(figsize=(15, 15))

In [5]:
from pregroupFunctorMappings import count_boxes, object_mapping, arrow_mapping

num_of_result_columns = count_boxes(diagram, "result-column")
num_of_result_columns += count_boxes(diagram, "result-column-with-alias")
num_of_tables = count_boxes(diagram, "table")
num_of_tables += count_boxes(diagram, "table-with-alias")

Rewriter = Functor(ob = lambda x: object_mapping(x, num_of_result_columns, num_of_tables), ar = lambda f: arrow_mapping(f, num_of_result_columns, num_of_tables))
pregroup_diagram = Rewriter(diagram)
width = pregroup_diagram.width()
height = pregroup_diagram.depth()
dim = 3*max(width, height)
total_dim += dim

In [6]:
print(dim)
pregroup_diagram.draw(figsize=(23, 22), fontsize = 17, fontsize_types = 17, scale = (0.5, 1), path = this_folder + "\\figures\\paper_figures\\pregroup_diagram_cat_example.tex", to_tikz = True)

27


## Convert pregroup representations to circuits

Following the ideas of the paper A Quantum Natural Language Processing Approach to Musical Intelligence, we can reduce the number of qubits by rewriting the diagram and removing cups.

An assumption is that every box contains a connection to a cup. On the other hand, the SELECT-box does not need to be changed. Thus for every box (which is not a cup or the select box) we "raise the first leg on top of the box". This process creates snakes which the normalization process automatically removes. This rewriting process ensures that the cups are removed from the diagram and we can use less qubits.

In [7]:
def cup_remove_arrow_mapping(box):
    if box.name.lower() == 'select':
        return box
    elif not box.cod:
        domain = box.dom
        raised_leg = Ty(domain[0])
        new_domain = reduce(lambda x, y : x @ Ty(y), domain[1:], Ty())
        new_box = Id(raised_leg) @ Box(box.name, new_domain, raised_leg.l)\
        >> Cup(raised_leg, raised_leg.l)
        return new_box
    return box

def cup_remove_arrow_mapping2(box):
    if box.cod == box.dom == Ty('n'):
        return Id(box.cod)
    return box

cup_removal_functor = Functor(ob = lambda x: x, ar = lambda f: cup_remove_arrow_mapping(f))
cup_removal_functor2 = Functor(ob = lambda x: x, ar = lambda f: cup_remove_arrow_mapping2(f))

In [8]:
cupless_pregroup_diagram = cup_removal_functor(pregroup_diagram.normal_form()).normal_form()
cupless_pregroup_diagram = cup_removal_functor2(cupless_pregroup_diagram).normal_form()
width = cupless_pregroup_diagram.width()
height = cupless_pregroup_diagram.depth()
dim = 3*max(width, height)
total_dim += dim

In [9]:
print(dim)
cupless_pregroup_diagram.draw(figsize=(dim, dim), fontsize = 17, fontsize_types = 17, scale = (1,1), path = this_folder + "\\figures\\paper_figures\\cup_removed_pregroup_diagram_cat_example.tex", to_tikz = True)

15


In [10]:
cup_removed_pregroup_folder_name = "join-order-benchmark-diagrams//cup-removed-pregroup-diagrams"
with open(this_folder + "\\" + cup_removed_pregroup_folder_name + "\\" + "4b" + ".json", 'w') as outfile:
    json.dump(json.loads(dumps(cupless_pregroup_diagram)), outfile)

### Pregroup grammar to circuit ansatz functor

In [11]:
n, s = Ty('n'), Ty('s')
ansatz = IQPAnsatzFlipped({n: 1, s: 1}, n_layers=1, n_single_qubit_params=1)
circuit_diagram = ansatz(cupless_pregroup_diagram)
width = circuit_diagram.width()
height = circuit_diagram.depth()
dim = 0.9*max(width, height)
total_dim += dim

In [12]:
print(dim)
circuit_diagram.draw(figsize=(15, 20), fontsize = 17, fontsize_types = 17, scale = (0.6,0.7), path = this_folder + "\\figures\\paper_figures\\circuit_diagram_cat_example.png")

15.3


## Draw process in single figure

In [13]:
from discopy.drawing import equation

equation_diagram = equation(diagram, 
                            cupless_pregroup_diagram, 
                            circuit_diagram, 
                            symbol = '→', 
                            fontsize = 12,
                            figsize=(23, 9),
                            fontsize_types = 12, 
                            path = this_folder + "\\figures\\cat_equation.png",
                            scale = (1.2, 0.7),
                            pad = (0,0),
                            draw_type_labels = False)

## Transforming circuit in Pennylane

In [14]:
dev = qml.device("qiskit.aer", wires=6, backend='unitary_simulator')

#symbols = set([elem for c in all_circuits for elem in all_circuits[c].free_symbols])
symbols = list(sorted(circuit_diagram.free_symbols, key=default_sort_key))

pennylane_circuit = to_pennylane(circuit_diagram)
params = pennylane_circuit.params
print(params)

ops = pennylane_circuit.ops
param_symbols = [[sym[0].as_ordered_factors()[1]] if len(sym) > 0 else [] for sym in params]
pennylane_wires = pennylane_circuit.wires

circuit_elements = reversed(list(zip(ops, param_symbols, pennylane_wires)))
print(circuit_elements)

symbol_to_index = {}
for sym in param_symbols:
    if len(sym) > 0:
        symbol_to_index[sym[0]] = symbols.index(sym[0])

@qml.qnode(dev)
def qml_circuit(circ_params):
    for op, param, wires in circuit_elements:
        if len(param) > 0:
            param = param[0]
            op(circ_params[symbol_to_index[param]], wires = wires)
        else:
            op(wires = wires)
    return qml.sample()

[[6.28318530717959*WHERE_s@n.l_n_0], [6.28318530717959*favourite_food__n.l_0], [6.28318530717959*cat_name__n.l_0], [6.28318530717959*cats__n.l_0], [6.28318530717959*'Whiskers'__n.l_0], [6.28318530717959*cat_name__n.l_0], [6.28318530717959*FROM_n@n.l_n_1], [6.28318530717959*=_n.l@n.l_n.l_0], [6.28318530717959*=_n.l@n.l_n.l_1], [6.28318530717959*=_n.l@n.l_n.l_0], [], [], [6.28318530717959*WHERE_s@n.l_n_1], [6.28318530717959*WHERE_s@n.l_n_0], [], [], [6.28318530717959*FROM_n@n.l_n_0], [6.28318530717959*FROM_n@n.l_n_0], [], [], [6.28318530717959*SELECT_n@n.l@n.l__0], [], [6.28318530717959*SELECT_n@n.l@n.l__1], [], []]


In [15]:

params = [2*np.pi*random.uniform(0, 1) for i in range(len(symbols))]
qml_circuit(params)
print(dev._circuit.draw(output = "latex_source"))

\documentclass[border=2px]{standalone}

\usepackage[braket, qm]{qcircuit}
\usepackage{graphicx}

\begin{document}
\scalebox{1.0}{
\Qcircuit @C=1.0em @R=0.2em @!R { \\
	 	\nghost{{q}_{0} :  } & \lstick{{q}_{0} :  } & \gate{\mathrm{H}} & \qw & \ctrl{1} & \gate{\mathrm{H}} & \ctrl{3} & \gate{\mathrm{R_X}\,(\mathrm{0.5257})} & \gate{\mathrm{H}} & \ctrl{4} & \gate{\mathrm{R_X}\,(\mathrm{0.6727})} & \qw & \qw & \qw & \qw & \qw & \qw\\
	 	\nghost{{q}_{1} :  } & \lstick{{q}_{1} :  } & \gate{\mathrm{H}} & \ctrl{1} & \gate{\mathrm{R_Z}\,(\mathrm{4.471})} & \gate{\mathrm{R_X}\,(\mathrm{4.255})} & \qw & \qw & \qw & \qw & \qw & \qw & \qw & \qw & \qw & \qw & \qw\\
	 	\nghost{{q}_{2} :  } & \lstick{{q}_{2} :  } & \gate{\mathrm{H}} & \gate{\mathrm{R_Z}\,(\mathrm{0.2589})} & \gate{\mathrm{R_X}\,(\mathrm{1.956})} & \qw & \qw & \qw & \qw & \qw & \qw & \qw & \qw & \qw & \qw & \qw & \qw\\
	 	\nghost{{q}_{3} :  } & \lstick{{q}_{3} :  } & \gate{\mathrm{H}} & \qw & \qw & \qw & \gate{\mathrm{R_Z}\,(\mathrm{0.5